<a href="https://colab.research.google.com/github/beatricekiplagat/Deepfake-Audio-Recognition/blob/main/DeepFake_Audio_Detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEPFAKE AUDIO DETECTION

Workflow
1. Load the dataset 
2. preprocessing - listen to the samples (real and spoofed)
3. convert into spectrogram then to mel spectrogram 
4. choose a suitable sample
5. create a model 
- Pre-Trained Models for Image Classification
VGG-16 - research on all of these
ResNet50
Inceptionv3
EfficientNet
6. Fine tune the model
6. deploy it on website

 

# Importing Libraries

In [1]:
!pip install wandb
!pip install playsound

     |████████████████████████████████| 1.7 MB 7.1 MB/s 
     |████████████████████████████████| 133 kB 48.7 MB/s 
     |████████████████████████████████| 97 kB 5.1 MB/s 
     |████████████████████████████████| 170 kB 52.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=5b7076baea0835de576918b7f9dbd6d00894acaa6ba86bb2e6670508bb828353
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=caaa4e73d55a3b6ae61bec51828234be878cc30201029c5a98464eb18aa85532
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=878b17c7aefbc1128975dba6d038c0801234b65864c474058e3bd

In [4]:
import numpy as np 
import pandas as pd
import os
import glob
import soundfile as sf
import scipy.io.wavfile
import scipy.signal as signal
import matplotlib.pyplot as plt
import pathlib
import shutil
import random
import wandb
from IPython.display import Audio
from scipy.io import wavfile
from playsound import playsound

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


# Load the Data

In [ ]:
# Unzipping the file 
!unzip "/content/drive/MyDrive/projectweeks/LA.zip" 

In [5]:
# Initialize wandb
wandb.login()
wandb.init(project='DeepFake Audio Detection', entity='ruoro')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ruoro (use `wandb login --relogin` to force relogin)


In [6]:
# Checking the working directory 
os.getcwd()
# Changing the Working Directory 
os.chdir("/content/LA")
# Confimation
os.getcwd()

'/content/LA'

In [7]:
# Create the dataset 
df = pd.read_csv("/content/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt", sep = " ", names =  ("a","id","c","fakeclass","class"))
df.head()

,a,id,c,fakeclass,class
0,LA_0069,LA_D_1047731,-,-,bonafide
1,LA_0069,LA_D_1105538,-,-,bonafide
2,LA_0069,LA_D_1125976,-,-,bonafide
3,LA_0069,LA_D_1293230,-,-,bonafide
4,LA_0069,LA_D_1340209,-,-,bonafide


That means there is a lot of dissimilarity. But we need to separate the data ie the first bonafide and the spoof class


In [8]:
df1 = df[df['fakeclass']== "-"]
df1.shape

(2548, 5)

In [9]:
# so now that df1 is only the bonified data we can get 

In [10]:
# Move all the real audio files in one location
pathlib.Path(f'AudioSample/real').mkdir(parents = True ,exist_ok = True)
real_audio = df1['id']

#
for file in real_audio:
  file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/"+ file +".flac" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/LA/AudioSample/real')


In [11]:
# log the real audio samples on wandb 
# for loop 

In [12]:
# Sample . 

In [13]:
df3 = df[df['fakeclass'].isin(['A05', 'A06', 'A17', 'A18', 'A19'])]
df3.shape

(7432, 5)

In [14]:
# Move all the Vocoder synthesized audio files in one location
pathlib.Path(f'AudioSample/voc').mkdir(parents = True ,exist_ok = True)
name_of_audio = df3['id']

#
for file in name_of_audio:
  file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/", file +".flac" )
  if os.path.isfile(file_name):
    shutil.copy(file_name, 'AudioSample/voc')

In [15]:
# log inti wandb 


In [16]:
# sample

In [43]:
import plotly.express as px
import plotly.graph_objects as go

# 
# clas = px.df['class'].value_counts()
fig = go.Figure(data=[go.Pie(labels=['Spoof','Bonafide'],
                             values=[2548,22296])])
fig.update_traces(hoverinfo='label+percent', textinfo='value+label', textfont_size=15,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

There is a lot of Data that is spoofed so we will have to segment it. 

In [34]:
# Finding out the number of classes in the data
df['class'].value_counts()

spoof       22296
bonafide     2548
Name: class, dtype: int64

In [46]:
fakeclass = df['fakeclass'].value_counts()
fakeclass.to_dict()

{'-': 2548,
 'A01': 3716,
 'A02': 3716,
 'A03': 3716,
 'A04': 3716,
 'A05': 3716,
 'A06': 3716}

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Pie(fakeclass.to_dict)])
#fig.update_traces(hoverinfo='label+percent', textinfo='value+label', textfont_size=15,
 #                 marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [22]:
Audio("/content/LA/AudioSample/real/LA_D_1024892.flac")